In [ ]:
using NBInclude, NLopt
_PRECOMPILE_=true
_LC_=true
@nbinclude("startup_GER.ipynb")

# 1) Find the parameters of the benchmark model

We calibrate our model to determine the values of $\alpha$, $m$ and $q_A$.

In [ ]:
myq0 = 0.02

function distance(m1,m2)
    out = 0
    for i in 1:length(m1)
        out += (m1[i] - m2[i]) ^2
    end
    return(out)
end

## compute the objective to minimise
function objective_ger(α,m,q0,qa)
    newC = update_Country2(α=α, m=m, q0=q0, qa=qa, λ0=1/12, λ1=1/18)

    ## simulate
    V = find_Surplus(newC,reltol=1e-12, findΨ=false)
    moments_emp = moments_germany(V,newC)
    
    ## printed info
  #  IJulia.clear_output(true)
  #  println("Current parameters: $α, $m, $q0, $qa")
    
    out = distance(moments_emp, moments_target) 
  #  print("$out - ")
    out
end

In [ ]:
## distance minimisation with NLopt (/!\ time-consuming)
@time parmin = let
    opt = Opt(:LN_BOBYQA, 4)
    lower_bounds!(opt, [0.005 , 0., myq0, -0.001])
    upper_bounds!(opt, [2. , 1., myq0, 0.001])
    xtol_rel!(opt,1e-12) 
    maxtime!(opt, 3600)
    min_objective!(opt, (v,g)-> objective_ger(v[1],v[2],v[3],v[4]))

    init = [0.8, 0.1, myq0, 0.]

    (minf,minx,ret) = optimize(opt, init) 
    println("Parameters found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    println(" - λ0= ", 1/12)
    println(" - λ1= ", 1/18)
    minx
end
;

# 2) Sensitivity: find the parameters with alternative values of $q_0$
Find the parameters for $q_0=0.005, 0.01, 0.025, 0.05, 0.1$.

In [ ]:
function objective_ger2(α,m,q0,qa)
    try objective_ger(α,m,q0,qa)
    catch
        Inf
    end
end


function find_germany(q0; init = [0.8, 0.1, 0.])
    opt = Opt(:LN_SBPLX, 3)
    lower_bounds!(opt, [0.005 , 0., -0.0005])
    upper_bounds!(opt, [3. , 1., 0.0005])
    xtol_rel!(opt,1e-12)
    maxtime!(opt, 12*3600)
    min_objective!(opt, (v,g)-> objective_ger2(v[1],v[2],q0, v[3]))

    (minf,minx,ret) = optimize(opt, init) 
    println("Parameters found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", q0)
    println(" - qa= ", minx[3])
    return(minx, minf)
end

q0_vec = [0.01 0.025 0.05 0.075 0.1]
#[0.005 0.01 0.025 0.05 0.1]
amqa_vec = zeros(3,5)
objectivevalue = zeros(5)
let
   @time  amqa_vec[:,1], objectivevalue[1]  =  find_germany(q0_vec[1], init = [0.7, 0.09, 1e-5])
   @time  amqa_vec[:,2], objectivevalue[2]  =  find_germany(q0_vec[2], init = [1.1, 0.1, 3e-5])
   @time  amqa_vec[:,3], objectivevalue[3]  =  find_germany(q0_vec[3], init = [1.7, 0.13, 8e-5])
   @time  amqa_vec[:,4], objectivevalue[4]  =  find_germany(q0_vec[4], init = [2.3, 0.13, 1e-4]) 
   @time  amqa_vec[:,5], objectivevalue[5]  =  find_germany(q0_vec[5], init = [2.9, 0.13, 2e-4]) 
end

println(objectivevalue)
out = vcat(q0_vec,amqa_vec)
show(out)
out